In [30]:
import sys
import os
import gzip
import json
import pandas as pd
from collections import Counter, defaultdict
import multiprocessing as mp
import pysam
from pyBioInfo.Range import GRange
from pyBioInfo.Utils import ShiftLoader

In [2]:
# Load SVs from VCF file

def load_svs(path):
    svs = []
    with pysam.VariantFile(path) as f:
        for record in f:
            if record.contig == "chrY":
                continue
            svtype = record.info["SVTYPE"]
            if svtype != "DEL" and svtype != "INS":
                continue
            svlen = abs(record.info["SVLEN"])
            sv = GRange(chrom=record.contig, start=record.start, end=record.stop, name=record.id)
            sv.record = record
            sv.svtype = svtype
            sv.svlen = svlen
            svs.append(sv)
    return svs
    
f_vcf1 = "../../3_NanoStrandSeq_PseudoBulk/results/sv/filtered/PacBio.full.vcf.gz"
f_vcf2 = "../../4_NanoStrandSeq_Phasing/results/HG001_Cell_350/sv/cutesv.filtered.vcf.gz"
svs_ref = load_svs(f_vcf1)
svs_que = load_svs(f_vcf2)
print("Number of SV (reference):", len(svs_ref))
print("Number of SV (query):", len(svs_que))

[W::hts_idx_load3] The index file is older than the data file: ../../4_NanoStrandSeq_Phasing/results/HG001_Cell_350/sv/cutesv.filtered.vcf.gz.tbi


Number of SV (reference): 34137
Number of SV (query): 419873


In [4]:
# filter SV by supported reads

f_quant1 = "../../3_NanoStrandSeq_PseudoBulk/results/sv/quantify_lite/PacBio.full.tsv"
f_quant2 = "../../4_NanoStrandSeq_Phasing/results/HG001_Cell_350/sv/quantify_merged.tsv"

max_length = 10000
min_length = 50
min_reads = 5
min_freq = 0.2
min_query_cell = 3

dat = pd.read_csv(f_quant1, sep="\t")
dat["AgreeReadRatio"] = dat["AgreeRead"] / (dat["AgreeRead"] + dat["DisagreeRead"])
dat = dat[dat["Length"] <= max_length]
dat = dat[dat["Length"] >= min_length]
dat = dat[dat["Chrom"] != "chrY"]
dat = dat[dat["AgreeRead"] >= min_reads]
dat = dat[(dat["AgreeRead"] / (dat["AgreeRead"] + dat["DisagreeRead"])) >= min_freq]
ratios_ref = {name: ratio for name, ratio in dat[["Name", "AgreeReadRatio"]].values}
dat_ref = dat
names_ref = set(dat["Name"])

dat = pd.read_csv(f_quant2, sep="\t")
dat["AgreeReadRatio"] = dat["AgreeRead"] / (dat["AgreeRead"] + dat["DisagreeRead"])
dat = dat[dat["Length"] <= max_length]
dat = dat[dat["Length"] >= min_length]
dat = dat[dat["Chrom"] != "chrY"]
dat = dat[dat["AgreeRead"] >= min_reads]
if min_query_cell > 1:
    dat = dat[dat["AgreeCell"] >= min_query_cell]
dat = dat[(dat["AgreeRead"] / (dat["AgreeRead"] + dat["DisagreeRead"])) >= min_freq]
ratios_que = {name: ratio for name, ratio in dat[["Name", "AgreeReadRatio"]].values}
dat_que = dat
names_que = set(dat["Name"])

svs_ref_1 = list(filter(lambda sv: sv.name in names_ref, svs_ref))
svs_que_1 = list(filter(lambda sv: sv.name in names_que, svs_que))
print("Number of SV (reference):", len(svs_ref_1))
print("Number of SV (query):", len(svs_que_1))

Number of SV (reference): 19878
Number of SV (query): 21211


In [8]:
# Filter SVs by blacklist

def filter_sv_by_regions(svs, regions):
    svs1 = []
    loader = ShiftLoader(regions)
    for sv in svs:
        n = len(list(loader.fetch(obj=sv)))
        if n == 0:
            svs1.append(sv)
    return svs1

f_bed = "../../3_NanoStrandSeq_PseudoBulk/results/sv/regions/benchmark_sv_blacklist.bed"

regions = []
with open(f_bed) as f:
    for line in f:
        chrom, start, end = line.strip("\n").split("\t")
        start, end = int(start), int(end)
        regions.append(GRange(chrom=chrom, start=start, end=end))
regions.sort()

svs_ref_2 = filter_sv_by_regions(svs_ref_1, regions)
svs_que_2 = filter_sv_by_regions(svs_que_1, regions)
print("Number of SV (reference):", len(svs_ref_2))
print("Number of SV (query):", len(svs_que_2))

Number of SV (reference): 8294
Number of SV (query): 8643


In [6]:
# Assign GT

ref = svs_ref_2
que = svs_que_2

for sv in ref:
    if ratios_ref[sv.name] >= 0.8:
        sv.gt = "1/1"
    else:
        sv.gt = "0/1"
    
for sv in que:
    if ratios_que[sv.name] >= 0.8:
        sv.gt = "1/1"
    else:
        sv.gt = "0/1"

In [9]:
counter = Counter([sv.gt for sv in ref])
print(counter)
print("No. of heterozygous genomic variants:", counter["0/1"])

Counter({'0/1': 4558, '1/1': 3736})
No. of heterozygous genomic variants: 4558


In [32]:
# Assign phased GT

def get_read2hp(bamfile, chrom):
    d = dict()
    with pysam.AlignmentFile(bamfile) as f:
        for s in f.fetch(chrom):
            if s.has_tag("HP"):
                d[s.query_name] = int(s.get_tag("HP"))
    return d

path = "results/PacBio.read2hp.json"
if os.path.exists(path):
    read2hp = json.load(open(path))
else:
    results = []
    bamfile = "../../3_NanoStrandSeq_PseudoBulk/PacBio.full.haplotag.bam"
    pool = mp.Pool(23)
    with pysam.AlignmentFile(bamfile) as f:
        for chrom in f.references:
            r = pool.apply_async(get_read2hp, (bamfile, chrom))
            results.append(r)
    pool.close()
    pool.join()
    read2hp = dict()
    for r in results:
        d = r.get()
        for k, v in d.items():
            read2hp[k] = v
    with open(path, "w+") as fw:
        json.dump(read2hp, fw, indent=4)

In [15]:
m = pd.read_csv(gzip.open("../../3_NanoStrandSeq_PseudoBulk/results/sv/quantify/PacBio.full.tsv.gz"), sep="\t")

In [46]:
rows = []
for s in m["Detail"]:
    s = json.loads(s)
    row = []
    for k in ["Support", "Overlap", "Agree", "Disagree"]:
        hp1, hp2 = 0, 0
        for (read_name, read_group) in s[k]:
            hp = read2hp.get(read_name)
            if hp == 1:
                hp1 += 1
            elif hp == 2:
                hp2 += 2
        row.append(hp1)
        row.append(hp2)
    rows.append(row)

m2 = pd.DataFrame(rows, columns=["Support_HP1", "Support_HP2", "Overlap_HP1", "Overlap_HP2", "Agree_HP1", "Agree_HP2", "Disagree_HP1", "Disagree_HP2"])
m2["Name"] = m["Name"]
m2.head()

,Support_HP1,Support_HP2,Overlap_HP1,Overlap_HP2,Agree_HP1,Agree_HP2,Disagree_HP1,Disagree_HP2,Name
0,0,0,0,0,0,0,0,0,cuteSV.INS.0
1,0,0,0,0,0,0,0,0,cuteSV.INS.1
2,0,0,0,0,0,0,0,0,cuteSV.DEL.0
3,0,0,0,0,0,0,0,0,cuteSV.INS.2
4,0,0,0,0,0,0,0,0,cuteSV.DEL.1


In [51]:
d = dict()
for name, agree_hp1, agree_hp2, disagree_hp1, disagree_hp2 in m2[["Name", "Agree_HP1", "Agree_HP2", "Disagree_HP1", "Disagree_HP2"]].values:
    if agree_hp1 >= max(1, agree_hp2 * 4) and disagree_hp2 >= max(1, disagree_hp1 * 4):
        d[name] = "1|0"
    if agree_hp2 >= max(1, agree_hp1 * 4) and disagree_hp1 >= max(1, disagree_hp2 * 4):
        d[name] = "0|1"
for sv in ref:
    sv.gt_phased = d.get(sv.name)

In [52]:
m = pd.read_csv("results/quant_phased_svs.HG001_Cell_350.tsv", sep="\t", header=0)
d = dict()
for name, gt in m[["Name_HP1", "GenoType"]].values:
    if gt != ".":
        d[name] = gt
for sv in que:
    sv.gt_phased = d.get(sv.name)

In [53]:
def get_recall(svs_ref, svs_que, svtype):
    svs_ref = list(filter(lambda sv: sv.svtype == svtype, svs_ref))
    svs_que = list(filter(lambda sv: sv.svtype == svtype, svs_que))
    print("-" * 80)
    print(svtype, len(svs_ref), len(svs_que), sep="\t")
    print(Counter([sv.gt for sv in svs_ref]))
    
    n_hit = 0
    loader = ShiftLoader(svs_que)
    counter = defaultdict(int)
    for sv in svs_ref:
        hit = False
        for sv2 in loader.fetch(chrom=sv.chrom, start=sv.start - 1000, end=sv.end + 1000):
            if sv.svtype == sv2.svtype and min(sv.svlen, sv2.svlen) >= max(sv.svlen, sv2.svlen) * 0.7:
                sv.hit_gt_phased = sv2.gt_phased
                hit = True
                break
        if hit:
            counter[sv.gt] += 1
            n_hit += 1
    print(counter)
    data = dict()
    data["Reference"] = len(svs_ref)
    data["Query"] = len(svs_que)
    data["Reference_Hit"] = n_hit
    data["Reference_Recall"] = data["Reference_Hit"] / data["Reference"]
    return data


data = dict()

# deletion

d1 = get_recall(ref, que, "DEL")
d2 = get_recall(que, ref, "DEL")
recall = d1["Reference_Recall"]
precision = d2["Reference_Recall"]
f1 = 2 * recall * precision / (recall + precision)
data["Del_Recall"] = recall
data["Del_Precision"] = precision
data["Del_F1"] = f1
data["Del_Detail"] = [d1, d2]

# precision

d1 = get_recall(ref, que, "INS")
d2 = get_recall(que, ref, "INS")
recall = d1["Reference_Recall"]
precision = d2["Reference_Recall"]
f1 = 2 * recall * precision / (recall + precision)
data["Ins_Recall"] = recall
data["Ins_Precision"] = precision
data["Ins_F1"] = f1
data["Ins_Detail"] = [d1, d2]

--------------------------------------------------------------------------------
DEL	3160	3531
Counter({'0/1': 1846, '1/1': 1314})
defaultdict(<class 'int'>, {'1/1': 1296, '0/1': 1611})
--------------------------------------------------------------------------------
DEL	3531	3160
Counter({'0/1': 2154, '1/1': 1377})
defaultdict(<class 'int'>, {'1/1': 1251, '0/1': 1701})
--------------------------------------------------------------------------------
INS	5134	5112
Counter({'0/1': 2712, '1/1': 2422})
defaultdict(<class 'int'>, {'0/1': 2326, '1/1': 2094})
--------------------------------------------------------------------------------
INS	5112	5134
Counter({'0/1': 3112, '1/1': 2000})
defaultdict(<class 'int'>, {'0/1': 2601, '1/1': 1847})


In [67]:
d = dict()
for sv in ref:
    try:
        if sv.chrom not in d:
            d[sv.chrom] = [0, 0] # identical, not identical
        if sv.gt_phased and sv.hit_gt_phased:
            if sv.gt_phased == sv.hit_gt_phased:
                d[sv.chrom][0] += 1
            else:
                d[sv.chrom][1] += 1
    except AttributeError:
        continue
        
s1, s2 = 0, 0
for k, (v1, v2) in d.items():
    s1 += max(v1, v2)
    s2 += min(v1, v2)
print("Identical:", s1)
print("Not identical:", s2)

print("No. of phased genomic variants:", s1 + s2)
print("No. of phased genomic variants with REF:", s1, ", precision:", s1 / (s1 + s2))

Identical: 3128
Not identical: 10
No. of phased genomic variants: 3138
No. of phased genomic variants with REF: 3128 , precision: 0.9968132568514978
